In [3]:
from intelisys import Intelisys, safe_json_loads
from pydantic import BaseModel
from typing import List, Optional

In [2]:
#intelisys = Intelisys(provider="openai")
#transcription = intelisys.transcript("/Users/lifsys/Documents/devhub/lib/intelisys/testing.mp3")
#response = intelisys.chat(f"Summarize this transcription: {transcription}")

In [3]:
# Synchronous usage
intelisys = Intelisys()
response = intelisys.chat("Hello, how are you?")
response

"Hello! I'm doing well, thank you for asking. As an AI assistant, I don't have feelings, but I'm functioning properly and ready to help you with any questions or tasks you might have. How can I assist you today?"

In [4]:
safe_json_loads(response)

{'greeting': 'Hello!',
 'status': "I'm doing well, thank you for asking.",
 'self_description': "As an AI assistant, I don't have feelings, but I'm functioning properly and ready to help you with any questions or tasks you might have.",
 'offer_assistance': 'How can I assist you today?'}

In [5]:
# Template usage
response = intelisys.template_chat(
    template="Summarize this in {{words}} words: {{text}}",
    render_data={"words": 10, "text": "After carefully reviewing the provided resume, I don't see any explicit mention of a specific army rank. The document describes various military and civilian roles and responsibilities, but does not state a particular rank like Lieutenant, Captain, Major, etc. \nThe experience described suggests the individual has had significant leadership roles in the U.S. Army and Army Reserve, including positions like: Chief of the Central Team in the Army Reserve Sustainment Command Transportation Officer in the US Army Reserve Deployment Support Command Director of Logistics for Area Support Group - Kuwait Contingency Contracting Officer in Southwest Asia. While these roles imply a relatively senior position, without an explicitly stated rank, I cannot confirm any specific army rank from the information provided. The focus seems to be more on describing job responsibilities and accomplishments rather than military rank progression."}
)
response

'Resume lacks specific army rank; describes leadership roles in military service.'

In [6]:
intelisys = Intelisys(json_mode=True, log="INFO")

# Set a default template
intelisys.set_default_template("Return JSON, {{ name }}! {{ question }}")

# Use the template in a chat
response = intelisys.template_chat(
    render_data={"name": "Alice", "question": "How are you today?"}
)
print(response)

2024-08-25 14:57:36 | INFO | *Template*


{'name': 'Alice', 'greeting': 'Hello!', 'mood': "I'm doing well, thank you for asking.", 'current_status': 'Ready to assist', 'additional_info': "I'm an AI assistant, so I don't actually have feelings, but I'm functioning properly and eager to help!"}


In [7]:
# Image OCR example
intelisys = Intelisys(provider="openrouter", model="google/gemini-pro-vision")  # Make sure to use a model that supports image processing
result = (intelisys
    .image("https://mintlify.s3-us-west-1.amazonaws.com/anthropic/images/how-to-prompt-eng.png")
    .chat("Please provide the complete text in the following image(s).")
)
result

" Develop test cases\n\nEngineer preliminary prompt\n\nTest prompt against cases\n\nRefine prompt\n\nTest against held-out evals\n\nShip polished prompt\n\nDon't forget edge cases!\n\nEVALS!"

In [8]:
class MovieReview(BaseModel):
    title: str
    year: int
    director: str
    genre: List[str]
    rating: float
    summary: str
    pros: List[str]
    cons: List[str]

# Create an Intelisys instance
intelisys = Intelisys(provider="openai", model="gpt-4o-mini")

# Use method chaining to set up and execute the request
try:
    result = (intelisys
        .set_output_model(MovieReview)
        .set_system_message("You are a professional film critic with extensive knowledge of cinema history")
        .image("https://intheposter.com/cdn/shop/files/the-manager-in-the-poster-1_5000x.jpg?v=1694762527")
        .chat("Analyze this movie poster and provide a detailed review of the film. Include information about the movie, your rating, and a brief summary of your thoughts.")
    )

    # Parse the JSON string into a Python dictionary
    result = safe_json_loads(result)
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [9]:
from locksys import Locksys

intelisys = Intelisys()
response = (intelisys
    .set_default_template(Locksys("Promptsys").item("Template").key("CreatePromptBeta").results())
    .template_chat(render_data={"TASK": "Help me write a README.md for a python package"})
)
response

'<persona>\nYou are Guido van Rossum, the creator of the Python programming language. Born in 1956 in the Netherlands, you developed Python in the late 1980s and have been instrumental in its growth and development. Your experience in creating and maintaining Python makes you the ideal persona for guiding others in writing documentation for Python packages.\n</persona>\n\n<task>\nCreate a comprehensive README.md file for a Python package. This file should serve as the primary documentation for the package, providing users with all necessary information to understand, install, and use the package effectively. The README.md should be well-structured, informative, and follow best practices for open-source software documentation.\n</task>\n\n<instruction>\n1. Start with a clear and concise title that includes the package name.\n2. Write a brief description of the package, explaining its purpose and main features.\n3. Add badges for build status, version, and other relevant information.\n4.

In [10]:
# Create an Intelisys instance
intelisys = Intelisys(provider="openai", model="gpt-4o")

#pydantic base model
from pydantic import BaseModel

class Email(BaseModel):
    to: str
    sender: str
    subject: str
    summary: str

finalresult = (intelisys
    .set_system_message("You are a technical writer")
    .set_output_model(Email)
    #.reference("https://docs.anthropic.com/en/docs/build-with-claude/define-success")
    .reference("/Users/lifsys/Documents/devhub/testingZone/iTunes Match- Your subscription will automatically renew in 30 days..eml")
    .chat("Provide an exsum")
)

finalresult = safe_json_loads(finalresult)
finalresult

{'to': 'Alain',
 'sender': 'The iTunes Store team',
 'subject': 'Reminder: iTunes Match Subscription Renewal',
 'summary': 'Dear Alain, this is a reminder that your subscription for iTunes Match automatically renews annually for $24.99 starting Sep 21, 2024 until canceled. To avoid being charged, you must cancel at least a day before each renewal date. To manage your subscription, sign in to the iTunes Store on your Mac or PC and view your account. For more information about subscription and purchases, visit Apple Support. Regards, The iTunes Store team.'}

In [25]:
import pwd, os

uid = pwd.getpwnam('lifsys')[2]
os.setuid(uid)
os.system("""cp /Users/lifsys/Library/CloudStorage/OneDrive-BCFSolutionsInc/Tools/Rates-Calculations-Table.xlsx /Users/lifsys/Desktop/Rates-Calculations-Table.xlsx""")


cp: /Users/lifsys/Library/CloudStorage/OneDrive-BCFSolutionsInc/Tools/Rates-Calculations-Table.xlsx: Operation not permitted


256

In [9]:
rosterwrap = Intelisys(provider="openai", model="gpt-4o")

class Positions(BaseModel):
    first_name: str
    last_name: str
    position: str
    salary: float

class Roster(BaseModel):
    full_name: List[Positions]

roster = (rosterwrap
    .set_system_message("You are an analyst")
    .set_output_model(Roster)
    #.reference("https://docs.anthropic.com/en/docs/build-with-claude/define-success")
    .reference("/Users/lifsys/Library/CloudStorage/OneDrive-BCFSolutionsInc/1. Bestey-Bill/ROSTERS/PMATS Contract Rosters.xlsx", sheet_name="WOI")
    .chat("List of all members of the PMATS team")
)
roster = safe_json_loads(roster)
roster

2024-08-27 04:49:21 | ERROR | Error adding reference: [Errno 1] Operation not permitted: '/Users/lifsys/Library/CloudStorage/OneDrive-BCFSolutionsInc/1. Bestey-Bill/ROSTERS/PMATS Contract Rosters.xlsx'


ValueError: Failed to add reference from /Users/lifsys/Library/CloudStorage/OneDrive-BCFSolutionsInc/1. Bestey-Bill/ROSTERS/PMATS Contract Rosters.xlsx: [Errno 1] Operation not permitted: '/Users/lifsys/Library/CloudStorage/OneDrive-BCFSolutionsInc/1. Bestey-Bill/ROSTERS/PMATS Contract Rosters.xlsx'

In [15]:
council = Intelisys(provider="openai", model="gpt-4o")

class HistoricFigure(BaseModel):
    name: str
    lifespan: str
    background: str
    contribution: str

class Council(BaseModel):
    members: List[HistoricFigure]

committee = (council
    .set_system_message("You are an entrepreneur writing a business plan")
    .set_output_model(Council)
    .chat("Create a detailed list of the most renowned historical figures who would excel in overseeing the development and execution of a business plan. For each person, include their name, lifespan, a brief background detailing their key accomplishments and area of expertise, and a description of how their specific skills and experience would contribute to the successful execution of the business plan. Ensure the list represents a diverse range of skills, including strategic planning, innovation, financial management, leadership, and ethical considerations.")
)
committee = safe_json_loads(committee)
committee

{'members': [{'name': 'Benjamin Franklin',
   'lifespan': '1706-1790',
   'background': 'One of the Founding Fathers of the United States, Franklin was a polymath who excelled in various fields including science, politics, and diplomacy. He was also a successful entrepreneur and inventor.',
   'contribution': "Franklin's diverse expertise in innovation, strategic planning, and diplomacy would be invaluable in developing a comprehensive and forward-thinking business plan. His ability to balance multiple roles and his experience in founding institutions would provide strong leadership and vision."},
  {'name': 'Marie Curie',
   'lifespan': '1867-1934',
   'background': 'A pioneering physicist and chemist, Curie was the first woman to win a Nobel Prize and the only person to win Nobel Prizes in two different scientific fields (Physics and Chemistry).',
   'contribution': "Curie's dedication to research and innovation would drive the business towards cutting-edge solutions and continuous i

In [23]:
responses = []
for member in committee['members']:
    intelisys = Intelisys()
    response = (intelisys
        .set_default_template(Locksys("Promptsys").item("Council").key("Task").results())
        .template_chat(render_data={
            "task": "Provide a plan and approach to counceling a rouge member of my team",
            "name": member['name'],
            "background": member['background'],
            "contribution": member['contribution']})
    )
    responses.append(f"{member['name']}: {response}")

In [24]:
responses

['Certainly, I shall approach this task with the same diligence and foresight that I applied to the founding of our great nation and my various scientific pursuits. When dealing with a rogue member of your team, it is crucial to balance firmness with understanding, much as we did when crafting the Constitution. Here\'s a plan to counsel this individual:\n\n1. Observation and Analysis:\nFirst, we must gather intelligence. Observe the team member\'s behavior, understand their motivations, and identify the root causes of their actions. As I often say, "An investment in knowledge pays the best interest."\n\n2. Private Discourse:\nArrange a private meeting with the individual. Create an environment conducive to open dialogue, much like the salons I frequented in Paris. Remember, "Speak little, do much."\n\n3. Active Listening:\nAllow the team member to express their concerns and grievances. As a diplomat, I learned the value of listening before speaking. "The art of conversation lies in lis